In [0]:
import keras
import cv2
import numpy as np
from sklearn.metrics import accuracy_score
from keras.datasets import mnist
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.models import Sequential
from tqdm import tqdm

In [0]:
batch_size = 64
epochs = 3
IMAGE_WIDTH = 32
IMAGE_HEIGHT = 32
NUM_CLASSES = 10
NUM_MODELS = 10

# Preprocess

In [0]:
def preprocess(imgs):
    
    processed = []
    
    for img in tqdm(imgs):
        processed.append(cv2.resize(img, (IMAGE_WIDTH, IMAGE_HEIGHT), interpolation = cv2.INTER_AREA))
    
    proccessed = np.array(processed)
    return proccessed.reshape(proccessed.shape[0], IMAGE_WIDTH, IMAGE_HEIGHT, 1)

In [0]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = preprocess(x_train)
x_test = preprocess(x_test)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

100%|██████████| 10000/10000 [00:00<00:00, 133881.42it/s]

x_train shape: (60000, 32, 32, 1)
60000 train samples
10000 test samples


In [0]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)

In [0]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Model

In [0]:
def create_model():
    conv_base = VGG16(input_shape = (IMAGE_HEIGHT,IMAGE_WIDTH, 1),
                            include_top = False, weights = None, classes=NUM_CLASSES)

    conv_base.trainable = True

    model = Sequential()

    model.add(conv_base)
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(NUM_CLASSES))
    model.add(Activation('softmax'))

    optimizer = keras.optimizers.Adam(learning_rate=0.0001)

    model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['categorical_accuracy'])
    
    return model

# Train

In [0]:
models = []

for i in range(NUM_MODELS):

    print(f"Train model {i}")
    idx = np.random.choice(len(x_train), size=len(x_train), replace=True)

    x_train_model = x_train[idx]
    y_train_model = y_train[idx]

    model = create_model()
    model.fit(x_train_model,y_train_model,
              batch_size = batch_size,
              epochs = epochs,
              validation_data = (x_test,y_test),
              shuffle = True)
    models.append(model)

Train model 0
Train on 10000 samples, validate on 10000 samples
Epoch 1/3
10000/10000 [==============================] - 19s 2ms/step - loss: 1.1204 - categorical_accuracy: 0.5968 - val_loss: 0.3516 - val_categorical_accuracy: 0.8835
Epoch 2/3
10000/10000 [==============================] - 18s 2ms/step - loss: 0.2547 - categorical_accuracy: 0.9208 - val_loss: 0.1371 - val_categorical_accuracy: 0.9553
Epoch 3/3
10000/10000 [==============================] - 18s 2ms/step - loss: 0.1427 - categorical_accuracy: 0.9581 - val_loss: 0.1332 - val_categorical_accuracy: 0.9601
Train model 1
Train on 10000 samples, validate on 10000 samples
Epoch 1/3
10000/10000 [==============================] - 19s 2ms/step - loss: 1.1796 - categorical_accuracy: 0.5729 - val_loss: 0.3383 - val_categorical_accuracy: 0.8957
Epoch 2/3
10000/10000 [==============================] - 18s 2ms/step - loss: 0.2500 - categorical_accuracy: 0.9225 - val_loss: 0.1448 - val_categorical_accuracy: 0.9547
Epoch 3/3
10000/10000 

# Predict

In [0]:
prediction = []

for m in tqdm(models):
    prediction.append(np.argmax(m.predict(x_test), axis=1))

100%|██████████| 10/10 [00:52<00:00,  5.22s/it]


In [0]:
prediction = np.transpose(prediction)
prediction = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=1, arr=prediction)

print('Test accuracy:', accuracy_score(prediction, np.argmax(y_test, axis=1)))

Test accuracy: 0.9769
